<a href="https://colab.research.google.com/github/arturomf94/ce-mia/blob/master/sade/sade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install NiaPy==2.0.0rc5

     |████████████████████████████████| 194kB 2.8MB/s 
     |████████████████████████████████| 143kB 45.9MB/s 


In [0]:
import logging
import numpy as np

from NiaPy.algorithms.algorithm import Individual
from NiaPy.algorithms.basic.de import DifferentialEvolution, CrossBest1, CrossRand1, CrossCurr2Best1, CrossBest2, CrossCurr2Rand1, proportional, multiMutations, DynNpDifferentialEvolution
from NiaPy.util.utility import objects2array

logging.basicConfig()
logger = logging.getLogger('NiaPy.algorithms.modified')
logger.setLevel('INFO')

class SolutionSaDE(Individual):
	r"""Individual for SaDE algorithm.
	Attributes:
		F (float): Scale factor.
		CR (float): Crossover probability.
	See Also:
		:class:`NiaPy.algorithms.Individual`
	"""
	def __init__(self, F=2, CR=0.5, **kwargs):
		r"""Initialize SolutionSaDE.
		Attributes:
			F (float): Scale factor.
			CR (float): Crossover probability.
		See Also:
			:func:`NiaPy.algorithm.Individual.__init__`
		"""
		Individual.__init__(self, **kwargs)
		self.F, self.CR = F, CR

class SelfAdaptiveDifferentialEvolution(DifferentialEvolution):
	Name = ['SelfAdaptiveDifferentialEvolution', 'SaDE']

	@staticmethod
	def typeParameters():
		r"""Get dictionary with functions for checking values of parameters.
		Returns:
			Dict[str, Callable]:
		See Also:
			* :func:`NiaPy.algorithms.basic.DifferentialEvolution.typeParameters`
		"""
		d = DifferentialEvolution.typeParameters()
		return d

	def setParameters(self, **ukwargs):
		r"""Set the parameters of an algorithm.
		Arguments:
		See Also:
			* :func:`NiaPy.algorithms.basic.DifferentialEvolution.setParameters`
		"""
		DifferentialEvolution.setParameters(self, itype=ukwargs.pop('itype', SolutionSaDE), **ukwargs)
		if ukwargs: logger.info('Unused arguments: %s' % (ukwargs))

	def AdaptiveGen(self, x):
		r"""Adaptive update scale factor in crossover probability.
		Args:
			x (Individual): Individual to apply function on.
		Returns:
			Individual: New individual with new parameters
		"""
		f = np.random.normal(0.5, 0.3, 1)[0]
		cr = np.random.normal(.5, 0.1, 1)[0]
		return self.itype(x=x.x, F=f, CR=cr, e=False)

	def evolve(self, pop, xb, task, **ukwargs):
		r"""Evolve current population.
		Args:
			pop (numpy.ndarray[Individual]): Current population.
			xb (Individual): Global best individual.
			task (Task): Optimization task.
			ukwargs (Dict[str, Any]): Additional arguments.
		Returns:
			numpy.ndarray: New population.
		"""
		npop = objects2array([self.AdaptiveGen(e) for e in pop])
		for i, e in enumerate(npop): npop[i].x = self.CrossMutt(npop, i, xb, e.F, e.CR, rnd=self.Rand)
		for e in npop: e.evaluate(task, rnd=self.rand)
		return npop
  

In [54]:
# encoding=utf8
# This is temporary fix to import module from parent folder
# It will be removed when package is published on PyPI
import sys
sys.path.append('../')
# End of fix

#from NiaPy.algorithms.modified import SelfAdaptiveDifferentialEvolution
from NiaPy.task.task import StoppingTask, OptimizationType
from NiaPy.benchmarks import Sphere

# we will run SaDE algorithm for 5 independent runs
for i in range(5):
	task = StoppingTask(D=10, nGEN = 200, optType=OptimizationType.MINIMIZATION, benchmark=Sphere())
	algo = SelfAdaptiveDifferentialEvolution()
	best = algo.run(task=task)
	print('%s -> %s' % (best[0].x, best[1]))

# vim: tabstop=3 noexpandtab shiftwidth=3 softtabstop=3

[ 8.37196333e-04  7.91235124e-05 -1.80913283e-04 -1.62526046e-04
 -4.41481607e-04  2.15965910e-05  4.45613533e-04  6.01486025e-04
  3.58108107e-04  1.18604029e-04] -> 1.664340175814946e-06
[-2.55230962e-04 -6.97874611e-05  6.16433442e-05  1.54450335e-04
  2.14424715e-04  4.03136511e-04  5.96979602e-04 -2.94614477e-05
 -3.44063496e-04  2.49269356e-04] -> 8.439324693914702e-07
[ 4.39600661e-04  8.49969169e-04  1.39930966e-04  1.80407753e-04
 -3.04397649e-04 -1.89572979e-04  9.60441305e-05 -9.47287499e-05
  3.13332066e-04  1.87586609e-04] -> 1.247983535894232e-06
[-5.20669420e-05  7.20755660e-04 -4.66072329e-04 -6.22285845e-05
  3.98439600e-04  2.06632484e-04  3.17594260e-04 -1.89379983e-04
  3.83297979e-04  2.01531459e-04] -> 1.2690097610494477e-06
[ 3.17451983e-04  3.07234597e-05  2.36243554e-05  3.75081731e-04
  1.75533333e-04 -3.25510396e-04 -2.54582937e-04  2.28977947e-04
  2.30491007e-04  1.50889205e-04] -> 5.728701052662931e-07
